In [7]:
import renderapi
import renderapps
from renderapps.module.render_module import RenderModule, RenderParameters
import marshmallow as mm
from rtree import index as rindex
import networkx as nx
import json
from functools import partial

In [36]:
class RegisterTilePairClientParameters(RenderParameters):
    source_stack = mm.fields.Str(required=True,metadata={'description':'stack to find all tile matches from'})
    target_stack = mm.fields.Str(required=True,metadata={'desctiption':'stack to find tiles that are close to stack1'})
    merge_stack = mm.fields.Str(required=True,metadata={'desctiption':'stack that contains merge of source and target, defines z range to find match'})
    tile_radius = mm.fields.Float(required=False,default=.2,
                    metadata={'description':'fractions of a tile size to look for overlaps'})
    render_parameters_template = mm.fields.Str(required=False,
                                               default="{host}:{port}/owner/{owner}/project/{project}/stack/{stack}/tile/{id}/render-parameters",
                                               metadata={'description':'string template to use for tilepair file'})
    tile_pair_file = mm.fields.Str(required=True, metadata = {'description':'filepath to save tilepair file'})
class RegisterTilePairClient(RenderModule):
    def __init__(self,schema_type=None,*args,**kwargs):
        if schema_type is None:
            schema_type = RegisterTilePairClientParameters
        super(RegisterTilePairClient,self).__init__(schema_type=schema_type,*args,**kwargs)
    def run(self):
        self.logger.debug(json.dumps(self.args,indent=4))
        self.logger.error('WARNING NEEDS TO BE TESTED, TALK TO FORREST IF BROKEN')

        
        zvalues = renderapi.stack.get_z_values_for_stack(self.args['merge_stack'], render=self.render)
        mypartial = partial(self.find_pairs_in_z,
                            self.render,
                            self.args['source_stack'],
                            self.args['target_stack'],
                            self.args['tile_radius'])
        tilepairs = mypartial(zvalues[0])
        d = {}
        
        self.args['render'].update({'stack':self.args['merge_stack'],'id':'{id}'})
        print self.args['render']
        d['renderParametersUrlTemplate']=\
            self.args['render_parameters_template'].\
                 format(**self.args['render'])
        d['neighborPairs']=tilepairs
        
        with open(self.args['tile_pair_file'],'w') as fp:
            json.dump(d,fp)
                                                                   
        

    def find_pairs_in_z(self,render,source_stack,target_stack,radius,z):
        ridx = rindex.Index() #setup an Rtree to find overlapping tiles
        #get the tilespecs from both stacks for this z
        source_tiles = renderapi.tilespec.get_tile_specs_from_z(source_stack, z, render=render)
        target_tiles = renderapi.tilespec.get_tile_specs_from_z(target_stack, z, render=render)
        #insert the targets into an rtree
        [ridx.insert(i,(ts.minX,ts.minY,ts.maxX,ts.maxY)) for i,ts in enumerate(target_tiles)]

        tilepairs = []
        #loop over the source
        for ts in source_tiles:
            #find the targets that match
            nodes=list(ridx.intersection((ts.minX-ts.width*radius,
                                          ts.minY-ts.height*radius,
                                          ts.maxX+ts.width*radius,
                                          ts.maxY+ts.height*radius)))
            for node in nodes:
                ts_target=target_tiles[i]
                d = {
                    "p":{
                        "groupId":ts.layout.sectionId,
                        "id":ts.tileId
                    },
                    "q":{
                        "groupId":ts_target.layout.sectionId,
                        "id":ts_target.tileId
                    }
                }
                tilepairs.append(d)
        self.logger.debug('found {} pairs in layer {}'.format(len(tilepairs),z))
        return tilepairs 


In [37]:
renderargs={
        u"host":"em-131fs",
        u"port":8999,
        u"owner":"gayathri",
        u"project":"EM_Phase1",
        u"client_scripts":"/pipeline/render/render-ws-java-client/src/main/scripts"
    }

inputs = {
    "render":renderargs,
    "source_stack":"manual_align_Phase1Data_3464_3500_forrestfix",
    "target_stack":"Pinky40_20170313_aibsdata_flipped_shifted",
    "merge_stack":"Princeton_manual_align_testmerge",
    "log_level":"DEBUG",
    "tile_pair_file":"princeton_register_tilepair_v2.json"
}

mod = RegisterTilePairClient(input_data = inputs,args=[])
mod.run()

DEBUG:json_module:{
    "log_level": "DEBUG", 
    "render": {
        "project": "EM_Phase1", 
        "host": "em-131fs", 
        "memGB": "5G", 
        "owner": "gayathri", 
        "port": 8999, 
        "client_scripts": "/pipeline/render/render-ws-java-client/src/main/scripts"
    }, 
    "merge_stack": "Princeton_manual_align_testmerge", 
    "tile_radius": 0.2, 
    "render_parameters_template": "{host}:{port}/owner/{owner}/project/{project}/stack/{stack}/tile/{id}/render-parameters", 
    "source_stack": "manual_align_Phase1Data_3464_3500_forrestfix", 
    "target_stack": "Pinky40_20170313_aibsdata_flipped_shifted", 
    "tile_pair_file": "princeton_register_tilepair_v2.json"
}
ERROR:json_module:WARNING NEEDS TO BE TESTED, TALK TO FORREST IF BROKEN
DEBUG:json_module:found 16553 pairs in layer 3484.0


{'id': '{id}', 'stack': u'Princeton_manual_align_testmerge', 'project': u'EM_Phase1', 'host': u'em-131fs', 'memGB': u'5G', 'owner': u'gayathri', 'port': 8999, 'client_scripts': u'/pipeline/render/render-ws-java-client/src/main/scripts'}
